### Librerias

In [1]:
import Levenshtein

import string
import nltk
from nltk.corpus import stopwords  
from nltk.tokenize import word_tokenize, sent_tokenize

from lxml import html
import requests
import codecs

import gensim
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

## Ejercicio1

In [2]:
link_web_principal = 'https://dreguera.github.io/'
codigo_text_página = '//span[@id ="author"]/text()'

page = requests.get(link_web_principal)
tree = html.fromstring(page.content)
emisores = tree.xpath(codigo_text_página)

In [3]:
emisores

['Dani Reguera dreguera@mondragon.edu',
 'Carlos Cernuda ccernuda@mondragon.edu',
 'Dani Reguera dreguera@mondragon.edu',
 'Carlos Cernuda ccernuda@mondragon.edu',
 'Sara Segura spsegura@mondragon.edu',
 'Aitor Bediaga abediaga@mondragon.edu',
 'Sara Segura spsegura@mondragon.edu',
 'Carlos Cernuda ccernuda@mondragon.edu']

In [4]:
link_web_principal = 'https://dreguera.github.io/'
codigo_text_página = '//div[@id ="content"]/text()'

page = requests.get(link_web_principal)
tree = html.fromstring(page.content)
mensajes = tree.xpath(codigo_text_página)

In [5]:
#palabras2_tokenizado = [[w.lower() for w in word_tokenize(text)] 
#            for text in emisores]
#lista_palabras2 = palabras2_tokenizado[0]

In [6]:
#palabras2_tokenizado

In [7]:
#de="Dani"
#result=[]
#for m in emisores:
#        a = m.split(' ')[0]
#        if Levenshtein.distance(a ,de)==0:
#
#result.append(mensajes[0])
#print("El número de mensajes escritos por Dani, son:", result)

#### Usando la librería gensim

#### Apartado a

In [8]:
mensaje_1 = mensajes[2]

In [9]:
mensaje_2 = mensajes[0]

In [10]:
file_docs=[]
file_docs2=[]
tokens = sent_tokenize(mensaje_1)
for line in tokens:
    file_docs.append(line)
    
tokens = sent_tokenize(mensaje_2)
for line in tokens:
    file_docs2.append(line)

In [11]:
gen_docs = [[w.lower() for w in word_tokenize(text)] 
            for text in file_docs]

In [12]:
dictionary = gensim.corpora.Dictionary(gen_docs)
print(dictionary.token2id)

{'?': 0, 'clustering': 1, 'densidad': 2, 'habeis': 3, 'meanshift': 4, 'para': 5, 'por': 6, 'visto': 7, 'buenos': 8, 'da': 9, 'datos': 10, 'descartar': 11, 'me': 12, 'muy': 13, 'numã©ricos': 14, 'outliers': 15, 'resultados': 16, 'suelo': 17, 'utilizar': 18, 'y': 19, 'yo': 20}


In [13]:
corpus = [dictionary.doc2bow(gen_doc) for gen_doc in gen_docs]

In [14]:
tf_idf = gensim.models.TfidfModel(corpus)

In [15]:
import numpy as np
for doc in tf_idf[corpus]:
    print([[dictionary[id], np.around(freq, decimals=2)] for id, freq in doc])

[['?', 0.38], ['clustering', 0.38], ['densidad', 0.38], ['habeis', 0.38], ['meanshift', 0.38], ['por', 0.38], ['visto', 0.38]]
[['buenos', 0.28], ['da', 0.28], ['datos', 0.28], ['descartar', 0.28], ['me', 0.28], ['muy', 0.28], ['numã©ricos', 0.28], ['outliers', 0.28], ['resultados', 0.28], ['suelo', 0.28], ['utilizar', 0.28], ['y', 0.28], ['yo', 0.28]]


In [16]:
sims = gensim.similarities.Similarity('',tf_idf[corpus],
                                        num_features=len(dictionary))

In [17]:
for line in file_docs2:
    query_doc = [w.lower() for w in word_tokenize(line)]
    query_doc_bow = dictionary.doc2bow(query_doc)

In [18]:
query_doc_tf_idf = tf_idf[query_doc_bow]

In [19]:
print('Comparing Result:', sims[query_doc_tf_idf])

Comparing Result: [0.43643576 0.16012815]


In [20]:
sum_of_sims =(np.sum(sims[query_doc_tf_idf], dtype=np.float32))
print(sum_of_sims)

0.59656394


In [21]:
print(f'Average similarity percentage: {float(sum_of_sims / len(file_docs)) * 100}',"%")


Average similarity percentage: 29.82819676399231 %


#### apartado b

In [22]:
mensaje_3 = mensajes[1]

In [23]:
mensaje_4= mensajes[3]

In [24]:
file_docs=[]
file_docs2=[]
tokens = sent_tokenize(mensaje_3)
for line in tokens:
    file_docs.append(line)
    
tokens = sent_tokenize(mensaje_4)
for line in tokens:
    file_docs2.append(line)

In [25]:
gen_docs = [[w.lower() for w in word_tokenize(text)] 
            for text in file_docs]

In [26]:
dictionary = gensim.corpora.Dictionary(gen_docs)
corpus = [dictionary.doc2bow(gen_doc) for gen_doc in gen_docs]
tf_idf = gensim.models.TfidfModel(corpus)
for doc in tf_idf[corpus]:
    print([[dictionary[id], np.around(freq, decimals=2)] for id, freq in doc])

[['.', 0.22], ['centrado', 0.22], ['clustering', 0.22], ['dani', 0.22], ['de', 0.22], ['densidad', 0.22], ['el', 0.22], ['en', 0.44], ['he', 0.22], ['hola', 0.22], ['la', 0.22], ['me', 0.22], ['no', 0.22], ['parte', 0.22], ['por', 0.22], ['sobre', 0.22], ['supervisado', 0.22], ['todo', 0.22]]
[['dbscan', 0.41], ['hdscan', 0.41], ['hemos', 0.41], ['optics', 0.41], ['visto', 0.41], ['y', 0.41]]


In [27]:
sims = gensim.similarities.Similarity('',tf_idf[corpus],
                                        num_features=len(dictionary))

In [28]:
for line in file_docs2:
    query_doc = [w.lower() for w in word_tokenize(line)]
    query_doc_bow = dictionary.doc2bow(query_doc)

In [29]:
query_doc_tf_idf = tf_idf[query_doc_bow]

In [30]:
print('Comparing Result:', sims[query_doc_tf_idf])

Comparing Result: [0.19518001 0.5477226 ]


In [31]:
sum_of_sims =(np.sum(sims[query_doc_tf_idf], dtype=np.float32))
print(sum_of_sims)

0.7429026


In [32]:
print(f'Average similarity percentage: {float(sum_of_sims / len(file_docs)) * 100}',"%")

Average similarity percentage: 37.14512884616852 %


#### apartado c

In [33]:
mensaje_5 = mensajes[1]

In [34]:
mensaje_6= mensajes[4]

In [35]:
file_docs=[]
file_docs2=[]
tokens = sent_tokenize(mensaje_5)
for line in tokens:
    file_docs.append(line)
    
tokens = sent_tokenize(mensaje_6)
for line in tokens:
    file_docs2.append(line)

In [36]:
gen_docs = [[w.lower() for w in word_tokenize(text)] 
            for text in file_docs]

In [37]:
dictionary = gensim.corpora.Dictionary(gen_docs)
corpus = [dictionary.doc2bow(gen_doc) for gen_doc in gen_docs]
tf_idf = gensim.models.TfidfModel(corpus)
for doc in tf_idf[corpus]:
    print([[dictionary[id], np.around(freq, decimals=2)] for id, freq in doc])

[['.', 0.22], ['centrado', 0.22], ['clustering', 0.22], ['dani', 0.22], ['de', 0.22], ['densidad', 0.22], ['el', 0.22], ['en', 0.44], ['he', 0.22], ['hola', 0.22], ['la', 0.22], ['me', 0.22], ['no', 0.22], ['parte', 0.22], ['por', 0.22], ['sobre', 0.22], ['supervisado', 0.22], ['todo', 0.22]]
[['dbscan', 0.41], ['hdscan', 0.41], ['hemos', 0.41], ['optics', 0.41], ['visto', 0.41], ['y', 0.41]]


In [38]:
sims = gensim.similarities.Similarity('',tf_idf[corpus],
                                        num_features=len(dictionary))

In [39]:
for line in file_docs2:
    query_doc = [w.lower() for w in word_tokenize(line)]
    query_doc_bow = dictionary.doc2bow(query_doc)

In [40]:
query_doc_tf_idf = tf_idf[query_doc_bow]

In [41]:
print('Comparing Result:', sims[query_doc_tf_idf])

Comparing Result: [0.5819144 0.       ]


In [42]:
sum_of_sims =(np.sum(sims[query_doc_tf_idf], dtype=np.float32))
print(sum_of_sims)

0.5819144


In [43]:
print(f'Average similarity percentage: {float(sum_of_sims / len(file_docs)) * 100}',"%")

Average similarity percentage: 29.09572124481201 %


#### apartado d

In [44]:
mensajes

['He estado corrigiendo los trabajos de Data Science y la parte de aprendizaje no supervisado la han realizado bastante bien, Â¿QuÃ© algoritmos de clustering has dado en clase?',
 'Hola Dani, en la parte de no supervisado me he centrado sobre todo en el clustering por densidad. Hemos visto Optics, DBSCAN y HDSCAN',
 'MeanShift para clustering por densidad habeis visto? Yo suelo utilizar para datos numÃ©ricos y me da muy buenos resultados para descartar outliers',
 'Si, la verdad es que da buenos resultados pero cuando si puedes hacer ajustes manuales de otros parÃ¡metros, OPTICS y DBSCAN son mÃ¡s Ã³ptimos',
 'El tema del clutering por densidad me parece super interesante pero no se si tendrÃ\xada aplicabilidad en el Reto 7',
 'Creo que podrÃ\xadamos traer a una empresa para que les hablara de HDBSCAN',
 'Guay! me parece super interesante y puede ser super enriquecedor',
 'Perfecto']

In [45]:
vectorizer = CountVectorizer().fit_transform(mensajes)
vectors = vectorizer.toarray()

In [46]:
csim=cosine_similarity(vectors)

In [47]:
csim

array([[1.        , 0.40567404, 0.03194383, 0.16574839, 0.07377111,
        0.13093073, 0.        , 0.        ],
       [0.40567404, 1.        , 0.18898224, 0.15689291, 0.34914862,
        0.05163978, 0.06030227, 0.        ],
       [0.03194383, 0.18898224, 1.        , 0.1111874 , 0.12371791,
        0.14638501, 0.05698029, 0.        ],
       [0.16574839, 0.15689291, 0.1111874 , 1.        , 0.12838815,
        0.15191091, 0.        , 0.        ],
       [0.07377111, 0.34914862, 0.12371791, 0.12838815, 1.        ,
        0.        , 0.32897585, 0.        ],
       [0.13093073, 0.05163978, 0.14638501, 0.15191091, 0.        ,
        1.        , 0.        , 0.        ],
       [0.        , 0.06030227, 0.05698029, 0.        , 0.32897585,
        0.        , 1.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 1.        ]])

In [48]:
def cosine_sim_vectors(vec1,vec2):
    vec1 = vec1.reshape(1,-1)
    vec2 = vec2.reshape(1,-1)
    return cosine_similarity(vec1,vec2)[0][0]

In [49]:
cosine_sim_vectors(vectors[0],vectors[1]) #los que más se parecen

0.405674042269688

# ARIKETA2

In [50]:
import pandas as pd
df = pd.read_csv("Spotify-2000.csv")
df.columns

Index(['Index', 'Title', 'Artist', 'Top Genre', 'Year',
       'Beats Per Minute (BPM)', 'Energy', 'Danceability', 'Loudness (dB)',
       'Liveness', 'Valence', 'Length (Duration)', 'Acousticness',
       'Speechiness', 'Popularity'],
      dtype='object')

In [51]:
def combined_features(row):
    return row['Artist']+" "+row['Top Genre']
df["combinadas"] = df.apply(combined_features, axis =1)

In [52]:
vectorizer = CountVectorizer().fit_transform(df["combinadas"].values)
vectors = vectorizer.toarray()
csim = cosine_similarity(vectors)
csim

array([[1.  , 0.  , 0.  , ..., 0.  , 0.  , 0.5 ],
       [0.  , 1.  , 0.  , ..., 0.25, 0.  , 0.  ],
       [0.  , 0.  , 1.  , ..., 0.  , 0.  , 0.  ],
       ...,
       [0.  , 0.25, 0.  , ..., 1.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , ..., 0.  , 1.  , 0.  ],
       [0.5 , 0.  , 0.  , ..., 0.  , 0.  , 1.  ]])

In [53]:
indice = 31

In [54]:
canciones_similares = list(enumerate(csim[indice]))

In [55]:
canciones_similares_ordenadas =  sorted(canciones_similares, key=lambda x:x[1], reverse=True)
canciones_similares_ordenadas;

In [56]:
cinco_sim = canciones_similares_ordenadas[1:6]

In [57]:
cinco_sim

[(154, 0.5773502691896258),
 (231, 0.5773502691896258),
 (251, 0.5773502691896258),
 (441, 0.5773502691896258),
 (869, 0.5773502691896258)]

##### los 5 titulos

In [58]:
def get_title_from_index(Index):
    return df[df.Index == Index]["Title"].values[0]

contador = 0 
for cancion in cinco_sim: 
    print(get_title_from_index(cancion[0] ))
    contador += 1
    if contador == 5:
        break

Is Dit Alles
Feeling Good
Dochters
A Boy Named Sue (Live)
Storm And Thunder


##### los 5  autores

In [59]:
def get_author_from_title(Index):
    return df[df.Index == Index]["Artist"].values[0]

contador = 0 
for autor in cinco_sim: 
    print(get_author_from_title(autor[0]))
    contador += 1
    if contador == 5:
        break

Doe Maar
Michael Bublé
Marco Borsato
Johnny Cash
Earth & Fire
